<a href="https://colab.research.google.com/github/yamkas/Aggregation-in-SQL/blob/master/Aggregation_in_SQL_(for_student).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import sqlite3 as sql

In [ ]:
db = sql.connect('northwind.db')

In [ ]:
# query di bawah akan menampilkan tabel - tabel yang ada di database kita
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

In [ ]:
# buat variabel cursor dari koneksi ke database kita
cursor = db.cursor()

In [ ]:
# menjalankan query dan menampilkan hasil
cursor.execute(query)
print(cursor.fetchall())

[('Suppliers',), ('Shippers',), ('Employees',), ('Customers',), ('Orders',), ('Categories',), ('Products',), ('OrderDetails',)]


### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

In [ ]:
pd.read_sql("SELECT * FROM orders LIMIT 3;", db)

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2014-07-04 08:00:00.000,2014-08-01 00:00:00.000,2014-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,2014-07-05 04:00:00.000,2014-08-16 00:00:00.000,2014-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,2014-07-08 15:00:00.000,2014-08-05 00:00:00.000,2014-07-12 00:00:00.000,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil


In [ ]:
pd.read_sql("SELECT COUNT(*) FROM suppliers;", db)

,COUNT(*)
0,29


In [ ]:
pd.read_sql("SELECT UPPER(City), * FROM Suppliers LIMIT 3;", db)

,UPPER(City),SupplierID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,LONDON,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,UK,(171) 555-2222,None,None
1,NEW ORLEANS,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,None,#CAJUN.HTM#
2,ANN ARBOR,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,None


### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

In [ ]:
pd.read_sql("SELECT AVG(freight) FROM Orders;", db)

,AVG(freight)
0,80.627205


In [ ]:
pd.read_sql("SELECT AVG(freight), ShipCountry FROM Orders GROUP BY 2;", db)

,AVG(freight),ShipCountry
0,37.411250,Argentina
1,184.787500,Austria
2,67.375789,Belgium
3,58.797470,Brazil
4,73.269667,Canada
5,77.566111,Denmark
6,41.404091,Finland
7,80.723766,France
8,92.485902,Germany
9,145.012632,Ireland


In [ ]:
pd.read_sql("SELECT AVG(freight), ShipCountry, ShipName FROM Orders GROUP BY 2, 3;", db)

,AVG(freight),ShipCountry,ShipName
0,12.126667,Argentina,Cactus Comidas para llevar
1,61.328000,Argentina,Océano Atlántico Ltda.
2,43.836000,Argentina,Rancho grande
3,206.846333,Austria,Ernst Handel
4,118.611000,Austria,Piccolo und mehr
...,...,...,...
85,96.647143,USA,White Clover Markets
86,33.900000,Venezuela,GROSELLA-Restaurante
87,69.953333,Venezuela,HILARION-Abastos
88,52.457857,Venezuela,LILA-Supermercado


In [ ]:
pd.read_sql("SELECT AVG(freight), ShipCountry, COUNT(DISTINCT ShipName) FROM Orders GROUP BY 2;", db)

,AVG(freight),ShipCountry,COUNT(DISTINCT ShipName)
0,37.411250,Argentina,3
1,184.787500,Austria,2
2,67.375789,Belgium,2
3,58.797470,Brazil,9
4,73.269667,Canada,3
5,77.566111,Denmark,2
6,41.404091,Finland,2
7,80.723766,France,10
8,92.485902,Germany,12
9,145.012632,Ireland,1


In [ ]:
query_1 = """
SELECT AVG(freight)
      , MAX(freight)
      , MIN(freight)
      , ShipCountry
      , COUNT(DISTINCT ShipName)
      FROM Orders GROUP BY 4
"""
pd.read_sql_query(query_1, db)

,AVG(freight),MAX(freight),MIN(freight),ShipCountry,COUNT(DISTINCT ShipName)
0,37.411250,217.86,0.33,Argentina,3
1,184.787500,789.95,5.29,Austria,2
2,67.375789,424.30,0.17,Belgium,2
3,58.797470,890.78,0.14,Brazil,9
4,73.269667,379.13,0.94,Canada,3
5,77.566111,299.09,2.92,Denmark,2
6,41.404091,180.45,0.59,Finland,2
7,80.723766,2000.00,0.02,France,10
8,92.485902,1007.64,0.15,Germany,12
9,145.012632,603.54,16.74,Ireland,1


### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

In [ ]:
# It's an error to use WHERE to put a condition on an aggregated column.
pd.read_sql("SELECT AVG(freight) AS AVG_freight, ShipCountry FROM Orders GROUP BY 2 WHERE AVG_freight > 70;", db)

DatabaseError: Execution failed on sql 'SELECT AVG(freight) AS AVG_freight, ShipCountry FROM Orders GROUP BY 2 WHERE AVG_freight > 70;': near "WHERE": syntax error

In [ ]:
pd.read_sql("SELECT AVG(freight) AS AVG_freight, ShipCountry FROM Orders GROUP BY 2 HAVING AVG_freight > 70;", db)

,AVG_freight,ShipCountry
0,184.787500,Austria
1,73.269667,Canada
2,77.566111,Denmark
3,80.723766,France
4,92.485902,Germany
5,145.012632,Ireland
6,87.502703,Sweden
7,76.029444,Switzerland
8,112.879426,USA


In [ ]:
query_1 = """
SELECT AVG(freight) as AVG_freight
      , MAX(freight) as MAX_freight
      , MIN(freight) as MIN_freight
      , ShipCountry
      , COUNT(DISTINCT ShipName) as COUNT_shipname
      FROM Orders
      GROUP BY 4
      HAVING AVG_freight > 50
      AND COUNT_shipname > 2
"""
pd.read_sql_query(query_1, db)

,AVG_freight,MAX_freight,MIN_freight,ShipCountry,COUNT_shipname
0,58.797470,890.78,0.14,Brazil,9
1,73.269667,379.13,0.94,Canada,3
2,80.723766,2000.00,0.02,France,10
3,92.485902,1007.64,0.15,Germany,12
4,52.754821,288.43,0.90,UK,7
5,112.879426,830.75,0.20,USA,13
6,59.460435,210.19,0.12,Venezuela,4


# Mini Case

Selamat ! Kamu baru saja diterima bekerja sebagai seorang Jr. Data Analyst di perusahaan Nortwind Traders!. Hari ini adalah hari pertama kamu bekerja, dan manager kamu ingin kamu secara perlahan mengetahui seluk beluk dari perusahaan tempat kamu bekerja. Kamu diminta untuk mengerjakan beberapa tugas berikut menggunakan bantuan SQL yang sudah kamu pelajari sebelumnya.
Tugas kamu adalah membuat query untuk menjawab beberapa tugas yang akan diberikan di bawah ini, kemudian kamu dapat memberikan komentar sesuai dengan hasil query kamu.
Selamat bekerja!!

### 6.1. Menganalisa aktivitas pemesanan

In [ ]:
# tampilkan data pemesanan pada customer id VINET dengan tujuan negara France, dan memiliki nilai maksimal Freight lebih dari 10
query_4 = """
SELECT CustomerID, ShipCountry, MAX(Freight) FROM Orders
WHERE CustomerID = 'VINET'
AND ShipCountry = 'France'
GROUP BY 1 HAVING MAX(freight) > 10

"""
pd.read_sql_query(query_4, db)

,CustomerID,ShipCountry,MAX(Freight)
0,VINET,France,32.38


In [ ]:
# tampilkan data pemesanan dengan pengecualian --> ship region tidak boleh bernilai NONE
# dan diurutkan dari nilai freight tertinggi dengan rentang nilai rata-rata freight antara 30 sampai 35
query_5 = """
SELECT OrderID, ShipRegion, AVG(Freight) as AVGFreight FROM Orders
WHERE ShipRegion NOT LIKE '%NONE%'
GROUP BY OrderID
HAVING AVGFreight BETWEEN 30 AND 35
ORDER BY AVGFreight DESC

"""
pd.read_sql_query(query_5, db)

,OrderID,ShipRegion,AVGFreight
0,10357,Lara,34.88
1,10405,Nueva Esparta,34.82
2,10989,RJ,34.76
3,10383,Essex,34.24
4,10567,Co. Cork,33.97
5,11016,Essex,33.80
6,10685,SP,33.75
7,10936,OR,33.68
8,10913,SP,33.05
9,10975,BC,32.27


### 6.2 Mengidentifikasi Customer

In [ ]:
# tampilkan data jumlah customer di masing-masing country yang tinggal di Perancis dan USA
query_6 = """
SELECT CustomerID, ContactName, Country, COUNT(CustomerID)
FROM Customers
WHERE Country IN ('France', 'USA')
GROUP BY Country

"""
pd.read_sql_query(query_6, db)

,CustomerID,ContactName,Country,COUNT(CustomerID)
0,BLONP,Frédérique Citeaux,France,11
1,GREAL,Howard Snyder,USA,13


### 6.3 Info Product dan category

In [ ]:
# tampilkan info product berupa category id, harga termahal, harga termurah, dan diurutkan berdasarkan rata-rata stock dari terbesar ke terkecil
# kategori yang ditampilkan memiliki rata-rata stock lebih dari 40
query_7 = """
SELECT CategoryID, MAX(UnitPrice) as HargaTermahal, MIN(UnitPrice) as HargaTermurah, AVG(UnitsInStock)
FROM Products
GROUP BY 1
HAVING AVG(UnitsInStock) > 40
ORDER BY AVG(UnitsInStock)DESC
"""
pd.read_sql_query(query_7, db)

,CategoryID,HargaTermahal,HargaTermurah,AVG(UnitsInStock)
0,8,62.5,6.0,58.416667
1,1,263.5,4.5,46.583333
2,5,38.0,7.0,44.000000
3,2,43.9,10.0,42.250000


In [ ]:
query_7 = """
SELECT * FROM Products
"""
pd.read_sql_query(query_7, db)

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,0
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,0
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,0
